# IRP Integration Demo

This notebook showcases usage of the `irp_integration` module. 

### Imports & Setup

In [7]:
%load_ext autoreload
%autoreload 2
import json
from datetime import datetime
from zoneinfo import ZoneInfo
from helpers.irp_integration import IRPClient
from helpers.sqlserver import execute_query_from_file, display_result_sets, sql_file_exists

irp_client = IRPClient()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Configurations

In [8]:
# Metadata / Backend Config
server_name = "databridge-1"
connection_name = "DATABRIDGE"
peril = "USFL"
date_value = "202511"
cycle_type = "Quarterly"
edm_data_version = "22"
geohaz_version = "22.0"

# SQL Scripts Workspace EDM
workspace_edm = "WORKSPACE_EDM"

# MRI Import Config
import_file_prefix = f"Modeling_{date_value}_Moodys"
mapping_file_name = "mapping.json"

# Analysis Config
analysis_job_name = f"{peril} Analysis"
analysis_profile_name = f"DLM {peril} v23"
output_profile_name = "Patched Portfolio Level Only (EP, ELT, Stats)"
event_rate_scheme_name = "RMS 2025 Stochastic Event Rates"
analysis_tag_name = f"{peril}_TAG"

# Grouping Config
group_name = f"{date_value}_{peril}_Group"

# RDM Config
rdm_name = f"RM_RDM_{date_value}_QEM_USAP"

### Create EDM(s)

In [9]:
### EDM Config ###
peril = "USFL"

edm_config = [
    f"RM_EDM_{date_value}_{cycle_type}_{peril}"
]

# Create Job List
edm_jobs = []
for edm_name in edm_config:
    edm_jobs.append(
        {
            "edm_name": edm_name,
            "server_name": server_name
        }
    )
print("EDMs to be created:")
print(json.dumps(edm_jobs, indent=2))

EDMs to be created:
[
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "server_name": "databridge-1"
  }
]


In [10]:
# Submit Jobs
job_ids = irp_client.edm.submit_create_edm_jobs(edm_jobs)
print(json.dumps(irp_client.job.poll_risk_data_job_batch_to_completion(job_ids), indent=2))

Polling batch risk data job ids: 23148952
Polling batch risk data job ids: 23148952
Polling batch risk data job ids: 23148952
Polling batch risk data job ids: 23148952
Polling batch risk data job ids: 23148952
Polling batch risk data job ids: 23148952
Polling batch risk data job ids: 23148952
Polling batch risk data job ids: 23148952
[
  {
    "jobId": "23148952",
    "userName": "Partner - PremiumIQ key",
    "status": "FINISHED",
    "submittedAt": "2025-11-11T19:37:32.038Z",
    "startedAt": "2025-11-11T19:37:49Z",
    "endedAt": "2025-11-11T19:39:33Z",
    "name": "RM_EDM_202511_Quarterly_USFL",
    "type": "CREATE_EDM",
    "progress": 100,
    "priority": "medium",
    "entitlement": "RI-RISKMODELER",
    "resourceGroupId": "ba672cd9-3384-4cb3-9882-bbeb5f8478c6",
    "details": {
      "resources": [],
      "summary": "CREATE_EDM is successful"
    },
    "warnings": []
  }
]


### Create Portfolio(s)

In [11]:
### Portfolio Config ###

## Example for single Base Portfolio ##
# portfolio_config = [
#     {
#         "edm_name": f"RM_EDM_{date_value}_{cycle_type}_{peril}",
#         "portfolio_name": peril
#     }
# ]

## Example for multiple Base Portfolios ##
portfolio_config = [
    {
        "edm_name": f"RM_EDM_{date_value}_{cycle_type}_{peril}",
        "portfolio_name": "USFL_Other"
    },
    {
        "edm_name": f"RM_EDM_{date_value}_{cycle_type}_{peril}",
        "portfolio_name": "USFL_Excess"
    },
    {
        "edm_name": f"RM_EDM_{date_value}_{cycle_type}_{peril}",
        "portfolio_name": "USFL_Commercial"
    }
]

# Create Job List
portfolio_jobs = []
for portfolio in portfolio_config:
    portfolio_jobs.append(
        {
            "edm_name": portfolio['edm_name'],
            "portfolio_name": portfolio['portfolio_name'],
            "portfolio_number": portfolio['portfolio_name'],
            "description": f"{portfolio['portfolio_name']} created via IRP Notebook Framework"
        }
    )
print("Portfolios to be created:")
print(json.dumps(portfolio_jobs, indent=2))

Portfolios to be created:
[
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "portfolio_name": "USFL_Other",
    "portfolio_number": "USFL_Other",
    "description": "USFL_Other created via IRP Notebook Framework"
  },
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "portfolio_name": "USFL_Excess",
    "portfolio_number": "USFL_Excess",
    "description": "USFL_Excess created via IRP Notebook Framework"
  },
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "portfolio_name": "USFL_Commercial",
    "portfolio_number": "USFL_Commercial",
    "description": "USFL_Commercial created via IRP Notebook Framework"
  }
]


In [12]:
portfolio_ids = irp_client.portfolio.create_portfolios(portfolio_jobs)
print(f"Created {len(portfolio_ids)} portfolios with IDs: {portfolio_ids}")

Created 3 portfolios with IDs: [1, 2, 3]


### MRI Import

In [14]:
### MRI Import Config ###
import_config = [
    {
        "portfolio_name": "USFL_Other",
        "edm_name": f"RM_EDM_{date_value}_{cycle_type}_{peril}",
        "import_file_value": "USFL_Other"
    },
    {
        "portfolio_name": "USFL_Excess",
        "edm_name": f"RM_EDM_{date_value}_{cycle_type}_{peril}",
        "import_file_value": "USFL_Excess"
    },
    {
        "portfolio_name": "USFL_Commercial",
        "edm_name": f"RM_EDM_{date_value}_{cycle_type}_{peril}",
        "import_file_value": "USFL_Commercial"
    }
]

import_file_prefix = f"Modeling_{date_value}_Moodys"

mri_jobs = []
for data in import_config:
    job_data = {}
    job_data['edm_name'] = data['edm_name']
    job_data['portfolio_name'] = data['portfolio_name']
    job_data['accounts_file_name'] = f"{import_file_prefix}_{data['import_file_value']}_Account.csv"
    job_data['locations_file_name'] = f"{import_file_prefix}_{data['import_file_value']}_Location.csv"        
    job_data['mapping_file'] = mapping_file_name
    mri_jobs.append(job_data)

print("MRI Data to be imported:")
print(json.dumps(mri_jobs, indent=2))

MRI Data to be imported:
[
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "portfolio_name": "USFL_Other",
    "accounts_file_name": "Modeling_202511_Moodys_USFL_Other_Account.csv",
    "locations_file_name": "Modeling_202511_Moodys_USFL_Other_Location.csv",
    "mapping_file": "mapping.json"
  },
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "portfolio_name": "USFL_Excess",
    "accounts_file_name": "Modeling_202511_Moodys_USFL_Excess_Account.csv",
    "locations_file_name": "Modeling_202511_Moodys_USFL_Excess_Location.csv",
    "mapping_file": "mapping.json"
  },
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "portfolio_name": "USFL_Commercial",
    "accounts_file_name": "Modeling_202511_Moodys_USFL_Commercial_Account.csv",
    "locations_file_name": "Modeling_202511_Moodys_USFL_Commercial_Location.csv",
    "mapping_file": "mapping.json"
  }
]


In [15]:
import_job_ids = irp_client.mri_import.import_from_files_batch(mri_jobs)
print(json.dumps(irp_client.mri_import.poll_import_job_batch_to_completion(import_job_ids), indent=2))


=== Processing import 1/3 ===
Looking up EDM: RM_EDM_202511_Quarterly_USFL
Looking up portfolio: USFL_Other
Creating AWS bucket...
AWS bucket created!
Uploading accounts file: Modeling_202511_Moodys_USFL_Other_Account.csv
Access credentials received
Uploading file /home/jovyan/workspace/workflows/_Tools/files/working_files/Modeling_202511_Moodys_USFL_Other_Account.csv to s3...
File uploaded!
Uploading locations file: Modeling_202511_Moodys_USFL_Other_Location.csv
Uploading file /home/jovyan/workspace/workflows/_Tools/files/working_files/Modeling_202511_Moodys_USFL_Other_Location.csv to s3...
File uploaded!
Uploading mapping file: mapping.json
Submitting import job for RM_EDM_202511_Quarterly_USFL/USFL_Other...
Import job submitted with workflow ID: 23149246

=== Processing import 2/3 ===
Looking up EDM: RM_EDM_202511_Quarterly_USFL
Looking up portfolio: USFL_Excess
Creating AWS bucket...
AWS bucket created!
Uploading accounts file: Modeling_202511_Moodys_USFL_Excess_Account.csv
Access

### Create Treaty

In [16]:
### Treaty Config ###

treaty_config = [
    {
        "edm_name": edm_name,
        "treaty_name": f"{peril}_Treaty",
        "treaty_number": f"{peril}_Treaty",
        "treaty_type": "Working Excess",
        "per_risk_limit": 30000000.00,
        "occurrence_limit": 90000000.00,
        "attachment_point": 2000000.00,
        "inception_date": "2025-01-01T00:00:00.000Z",
        "expiration_date": "2030-01-01T00:00:00.000Z",
        "currency_name": "US Dollar",
        "attachment_basis": "Losses Occurring",
        "attachment_level": "Location",
        "pct_covered": 100.0,
        "pct_placed": 95.00,
        "pct_share": 100.00,
        "pct_retention": 100.00,
        "premium": 0.00,
        "num_reinstatements": 99,
        "pct_reinstatement_charge": 0.00,
        "aggregate_limit": 0.00,
        "aggregate_deductible": 0.00,
        "priority": 1
    }
]

print("Treaties to be created:")
print(json.dumps(treaty_config, indent=2))

Treaties to be created:
[
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "treaty_name": "USFL_Treaty",
    "treaty_number": "USFL_Treaty",
    "treaty_type": "Working Excess",
    "per_risk_limit": 30000000.0,
    "occurrence_limit": 90000000.0,
    "attachment_point": 2000000.0,
    "inception_date": "2025-01-01T00:00:00.000Z",
    "expiration_date": "2030-01-01T00:00:00.000Z",
    "currency_name": "US Dollar",
    "attachment_basis": "Losses Occurring",
    "attachment_level": "Location",
    "pct_covered": 100.0,
    "pct_placed": 95.0,
    "pct_share": 100.0,
    "pct_retention": 100.0,
    "premium": 0.0,
    "num_reinstatements": 99,
    "pct_reinstatement_charge": 0.0,
    "aggregate_limit": 0.0,
    "aggregate_deductible": 0.0,
    "priority": 1
  }
]


In [17]:
treaty_ids = irp_client.treaty.create_treaties(treaty_config)
print(f"Created {len(treaty_ids)} with IDs: {treaty_ids}")

Created 1 with IDs: [1]


### Upgrade EDM Version

In [18]:
### Data Upgrade Config ###
upgrade_config = [
    {
        "edm_name": f"RM_EDM_{date_value}_{cycle_type}_{peril}",
        "edm_version": edm_data_version
    }
]

print("EDM Upgrades:")
print(json.dumps(upgrade_config, indent=2))

EDM Upgrades:
[
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "edm_version": "22"
  }
]


In [19]:
upgrade_job_ids = irp_client.edm.submit_upgrade_edm_data_version_jobs(upgrade_config)
print(json.dumps(irp_client.edm.poll_data_version_upgrade_job_batch_to_completion(upgrade_job_ids), indent=2))

Polling batch upgrade edm version job ids: 23149479
Polling batch upgrade edm version job ids: 23149479
Polling batch upgrade edm version job ids: 23149479
[
  {
    "id": 23149479,
    "userName": "Partner - PremiumIQ key",
    "status": "FINISHED",
    "submitTime": "2025-11-11T20:00:50.772Z",
    "startTime": "2025-11-11T20:00:51Z",
    "endTime": "2025-11-11T20:01:08Z",
    "name": "RM_EDM_202511_Quarterly_USFL",
    "type": "EDM_DATA_UPGRADE",
    "progress": 100,
    "priority": "medium",
    "entitlement": "RI-RISKMODELER",
    "resourceGroupId": "ba672cd9-3384-4cb3-9882-bbeb5f8478c6",
    "jobs": [
      {
        "id": "5acd7033-d310-42f0-81aa-bfb577da272d",
        "taskId": 1,
        "workflowId": 23149479,
        "status": "Succeeded",
        "submitTime": "2025-11-11T20:00:50.923Z",
        "createdAt": "2025-11-11T20:00:50.764Z",
        "name": "EDM_DATA_UPGRADE",
        "input": {
          "name": "EDM_DATA_UPGRADE"
        },
        "output": {
          "databas

### GeoHaz

In [20]:
### GeoHaz Config ###
geohaz_config = [
    {
        "edm_name": f"RM_EDM_{date_value}_{cycle_type}_{peril}",
        "portfolio_name": "USFL_Other"
    },
    {
        "edm_name": f"RM_EDM_{date_value}_{cycle_type}_{peril}",
        "portfolio_name": "USFL_Excess"
    },
    {
        "edm_name": f"RM_EDM_{date_value}_{cycle_type}_{peril}",
        "portfolio_name": "USFL_Commercial"
    }
]

geohaz_jobs = []
for data in geohaz_config:
    geohaz_jobs.append(
        {
            "edm_name": data['edm_name'],
            "portfolio_name": data['portfolio_name'],
            "version": geohaz_version,
            "hazard_eq": True,
            "hazard_ws": True
        }
    )

print("GeoHaz Job Info:")
print(json.dumps(geohaz_jobs, indent=2))

GeoHaz Job Info:
[
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "portfolio_name": "USFL_Other",
    "version": "22.0",
    "hazard_eq": true,
    "hazard_ws": true
  },
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "portfolio_name": "USFL_Excess",
    "version": "22.0",
    "hazard_eq": true,
    "hazard_ws": true
  },
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "portfolio_name": "USFL_Commercial",
    "version": "22.0",
    "hazard_eq": true,
    "hazard_ws": true
  }
]


In [21]:
geohaz_job_ids = irp_client.portfolio.submit_geohaz_jobs(geohaz_jobs)
print(json.dumps(irp_client.portfolio.poll_geohaz_job_batch_to_completion(geohaz_job_ids), indent=2))

Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 23149619,23149620,23149621
Polling batch geohaz job ids: 2

### Portfolio Mapping

In [22]:
### Mapping Config ###
mapping_config = [
    # {
    #     "edm_name": f"RM_EDM_{date_value}_{cycle_type}_USEQ",
    #     "portfolio_name": "USEQ",
    #     "import_file_value": "USEQ"
    # },
    {
        "edm_name": f"RM_EDM_{date_value}_{cycle_type}_USFL",
        "portfolio_name": "USFL_Other",
        "import_file_value": "USFL_Other"
    },
    {
        "edm_name": f"RM_EDM_{date_value}_{cycle_type}_USFL",
        "portfolio_name": "USFL_Excess",
        "import_file_value": "USFL_Excess"
    },
    {
        "edm_name": f"RM_EDM_{date_value}_{cycle_type}_USFL",
        "portfolio_name": "USFL_Commercial",
        "import_file_value": "USFL_Commercial"
    }
]

print("Portfolio Mapping:")
print(json.dumps(mapping_config, indent=2))

Portfolio Mapping:
[
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "portfolio_name": "USFL_Other",
    "import_file_value": "USFL_Other"
  },
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "portfolio_name": "USFL_Excess",
    "import_file_value": "USFL_Excess"
  },
  {
    "edm_name": "RM_EDM_202511_Quarterly_USFL",
    "portfolio_name": "USFL_Commercial",
    "import_file_value": "USFL_Commercial"
  }
]


In [23]:
for config in mapping_config:
    edm_name = config['edm_name']
    edms = irp_client.edm.search_edms(filter=f"exposureName=\"{edm_name}\"")
    if (len(edms) != 1):
        raise Exception(f"Found {len(edms)} EDMs with name {edm_name}; expected 1")
    edm = edms[0]
    exposure_id = edm['exposureId']
    edm_full_name = edm['databaseName']
    print(f'EDM Full Name: {edm_full_name}')

    portfolio_name = config['portfolio_name']
    portfolios = irp_client.portfolio.search_portfolios(exposure_id, f"portfolioName=\"{portfolio_name}\"")
    if (len(portfolios) != 1):
        raise Exception(f"Found {len(edms)} Portfolios with name {portfolio_name}; expected 1")
    portfolio_id = portfolios[0]['portfolioId']
    print(f'Portfolio ID: {portfolio_id}')

    sql_script_name = f"2b_Query_To_Create_Sub_Portfolios_{config['import_file_value']}_RMS_BackEnd.sql"
    sql_script_path = f"portfolio_mapping/{sql_script_name}"
    
    if not sql_file_exists(sql_script_path):
        print(f"SQL script {sql_script_path} does not exist")
        print(f"Skipping mapping for Portfolio {portfolio_name}")
        continue

    params = {
        "EDM_FULL_NAME": edm_full_name,
        "PORTFOLIO_ID": portfolio_id,
        "DATETIME_VALUE": datetime.now(ZoneInfo("America/New_York")).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    }

    result = execute_query_from_file(
        file_path = sql_script_path,
        params = params,
        connection = connection_name
    )

    display_result_sets(result)

EDM Full Name: RM_EDM_202511_Quarterly_USFL_LIjg
Portfolio ID: 1
Reading SQL script ...
Parameterizing ...
Executing SQL query: 2b_Query_To_Create_Sub_Portfolios_USFL_Other_RMS_BackEnd.sql
✓ Retrieved 1 result set(s)

QUERY RESULTS: 1 result set(s)


--------------------------------------------------------------------------------
Result Set 1 of 1
--------------------------------------------------------------------------------
Rows: 7 | Columns: 7



,PORTINFOID,PORTNUM,PORTNAME,CREATEDATE,DESCRIPT,AccountGroupCount,PortfolioAccountCount
0,5,USFL_Other_CB,USFL_Other_CB,2025-11-11 15:23:34.303,USFL_Other_CB,303,303
1,9,USFL_Other_Clay_21st,USFL_Other_Clay_21st,2025-11-11 15:23:34.303,USFL_Other_Clay_21st,68840,68840
2,10,USFL_Other_Clay_Home,USFL_Other_Clay_Home,2025-11-11 15:23:34.303,USFL_Other_Clay_Home,44770,44770
3,4,USFL_Other_Lender_P,USFL_Other_Lender_P,2025-11-11 15:23:34.303,USFL_Other_Lender_P,54701,54701
4,6,USFL_Other_Manufactu,USFL_Other_Manufactu,2025-11-11 15:23:34.303,USFL_Other_Manufactu,70307,70307
5,8,USFL_Other_Other,USFL_Other_Other,2025-11-11 15:23:34.303,USFL_Other_Other,214,214
6,7,USFL_Other_Renters,USFL_Other_Renters,2025-11-11 15:23:34.303,USFL_Other_Renters,1,1




EDM Full Name: RM_EDM_202511_Quarterly_USFL_LIjg
Portfolio ID: 2
SQL script portfolio_mapping/2b_Query_To_Create_Sub_Portfolios_USFL_Excess_RMS_BackEnd.sql does not exist
Skipping mapping for Portfolio USFL_Excess
EDM Full Name: RM_EDM_202511_Quarterly_USFL_LIjg
Portfolio ID: 3
SQL script portfolio_mapping/2b_Query_To_Create_Sub_Portfolios_USFL_Commercial_RMS_BackEnd.sql does not exist
Skipping mapping for Portfolio USFL_Commercial


### Control Totals

In [24]:
workspace_edms = irp_client.edm.search_edms(filter=f"exposureName=\"{workspace_edm}\"")
if (len(workspace_edms) != 1):
    raise Exception(f"Found {len(workspace_edms)} EDMs with name {workspace_edm}; expected 1")
workspace_edm = workspace_edms[0]
workspace_edm_full_name = workspace_edm['databaseName']
print(f'Workspace EDM Full Name: {workspace_edm_full_name}')
print()

Workspace EDM Full Name: WORKSPACE_EDM_uXJm



In [25]:
# EDM Control Totals

sql_script_name = '3d_RMS_EDM_CONTROL_Totals.sql'
params = {
    "WORKSPACE_EDM": workspace_edm_full_name,
    "CYCLE_TYPE": cycle_type,
    "DATE_VALUE": date_value
}

result = execute_query_from_file(
    file_path = f'control_totals/{sql_script_name}',
    params = params,
    connection = connection_name
)

display_result_sets(result)

Reading SQL script ...
Parameterizing ...
Executing SQL query: 3d_RMS_EDM_CONTROL_Totals.sql
✓ Retrieved 10 result set(s)

QUERY RESULTS: 10 result set(s)


--------------------------------------------------------------------------------
Result Set 1 of 10
--------------------------------------------------------------------------------
Rows: 7 | Columns: 6



,PORTNAME,PolicyCount,PolicyLimit,PolicyPremium,AttachmentPoint,PolicyDeductible
0,USEQ,516777,61568158484.00,565524244.00,0.00,0.00
1,USEQ_CHFS,344,362956599.00,1359514.00,0.00,0.00
2,USEQ_Clay_21st,148768,17232894141.00,158768016.00,0.00,0.00
3,USEQ_Clay_Homes,136315,19141446773.00,170120715.00,0.00,0.00
4,USEQ_Condo,144,21756768.00,149376.00,0.00,0.00
5,USEQ_Manufactured,196767,23973540757.00,227029441.00,0.00,0.00
6,USEQ_Renters,34439,835563446.00,8097182.00,0.00,0.00



--------------------------------------------------------------------------------
Result Set 2 of 10
--------------------------------------------------------------------------------
Rows: 7 | Columns: 3



,PORTNAME,LocationCountDistinct,LocationCountCampus
0,USEQ,516777,516777
1,USEQ_CHFS,344,344
2,USEQ_Clay_21st,148768,148768
3,USEQ_Clay_Homes,136315,136315
4,USEQ_Condo,144,144
5,USEQ_Manufactured,196767,196767
6,USEQ_Renters,34439,34439



--------------------------------------------------------------------------------
Result Set 3 of 10
--------------------------------------------------------------------------------
Rows: 7 | Columns: 3



...

,PORTNAME,LocationCountDistinct,LocationCountCampus
0,USFL_Commercial,10059,11487
1,USFL_Excess,191,210
2,USFL_Other,239136,239136
3,USFL_Other_CB,303,303
4,USFL_Other_Clayton,113610,113610
5,USFL_Other_Lender_P,54701,54701
6,USFL_Other_Manufactu,70307,70307
7,USFL_Other_Other,214,214
8,USFL_Other_Renters,1,1



--------------------------------------------------------------------------------
Result Set 9 of 10
--------------------------------------------------------------------------------
Rows: 9 | Columns: 4



,PORTNAME,TotalReplacementValue,LocationLimit,LocationDeductible_NonCommercialFlood
0,USFL_Commercial,30437380495.00,6212970382.00,164145750.00
1,USFL_Excess,2802870060.00,486544628.00,110000000.00
2,USFL_Other,33447594139.00,32333725460.00,188794250.00
3,USFL_Other_CB,29731001.00,27470350.00,0.00
4,USFL_Other_Clayton,15248131830.00,15248131830.00,56992250.00
5,USFL_Other_Lender_P,7984523506.00,6873319978.00,0.00
6,USFL_Other_Manufactu,10173184117.00,10173184117.00,131801750.00
7,USFL_Other_Other,12011685.00,11615685.00,0.00
8,USFL_Other_Renters,12000.00,3500.00,250.00



--------------------------------------------------------------------------------
Result Set 10 of 10
--------------------------------------------------------------------------------
Rows: 1 | Columns: 1



,USFL_Commercial_LocationDeductible
0,201230250.00


In [26]:
# GeoHaz Control Totals

sql_script_name = '3e_GeocodingSummary.sql'
params = {
    "WORKSPACE_EDM": workspace_edm_full_name,
    "CYCLE_TYPE": cycle_type,
    "DATE_VALUE": date_value
}

result = execute_query_from_file(
    file_path = f'control_totals/{sql_script_name}',
    params = params,
    connection = connection_name
)

display_result_sets(result, max_rows=100)

Reading SQL script ...
Parameterizing ...
Executing SQL query: 3e_GeocodingSummary.sql
✓ Retrieved 1 result set(s)

QUERY RESULTS: 1 result set(s)


--------------------------------------------------------------------------------
Result Set 1 of 1
--------------------------------------------------------------------------------
Rows: 15 | Columns: 7



,DBname,PORTNAME,GeoResolutionCode,GeocodeDescription,RiskCount,TIV,TRV
0,RM_EDM_202511_Quarterly_USEQ_DamQ,USEQ,0,Null,1,121940.00,121940.00
1,RM_EDM_202511_Quarterly_USEQ_DamQ,USEQ,1,Coordinate,482615,57104105908.00,57307871863.00
2,RM_EDM_202511_Quarterly_USEQ_DamQ,USEQ,2,Street address,13617,1720289855.00,1724018076.00
3,RM_EDM_202511_Quarterly_USEQ_DamQ,USEQ,4,Street Name,4198,557073685.00,557109685.00
4,RM_EDM_202511_Quarterly_USEQ_DamQ,USEQ,5,Postcode,16328,1976786566.00,1977427426.00
5,RM_EDM_202511_Quarterly_USEQ_DamQ,USEQ,7,City/Town,15,1609494.00,1609494.00
6,RM_EDM_202511_Quarterly_USFL_LIjg,USFL_Commercial,1,Coordinate,10056,6209416082.00,30434768993.00
7,RM_EDM_202511_Quarterly_USFL_LIjg,USFL_Commercial,2,Street address,3,3554300.00,2611502.00
8,RM_EDM_202511_Quarterly_USFL_LIjg,USFL_Excess,1,Coordinate,190,481644628.00,2793669869.00
9,RM_EDM_202511_Quarterly_USFL_LIjg,USFL_Excess,2,Street address,1,4900000.00,9200191.00


### Execute Multiple Analyses

In [ ]:
analysis_job_ids = irp_client.analysis.submit_portfolio_analysis_jobs(
    [
        {
            "edm_name": edm_name,
            "job_name": "CBHU 2023 Rates",
            "portfolio_name": portfolio_name,
            "analysis_profile_name": analysis_profile_name,
            "output_profile_name": output_profile_name,
            "event_rate_scheme_name": "RMS 2023 Stochastic Event Rates",
            "treaty_names": [treaty_name],
            "tag_names": [analysis_tag_name]
        },
        {
            "edm_name": edm_name,
            "job_name": "CBHU 2025 Rates",
            "portfolio_name": portfolio_name,
            "analysis_profile_name": analysis_profile_name,
            "output_profile_name": output_profile_name,
            "event_rate_scheme_name": "RMS 2025 Stochastic Event Rates",
            "treaty_names": [treaty_name],
            "tag_names": [analysis_tag_name]
        }
    ]
)
analysis_batch_response = irp_client.analysis.poll_analysis_job_batch_to_completion(analysis_job_ids)
print(json.dumps(analysis_batch_response, indent=2))

### Group Analyses

In [ ]:
analysis_names = ["CBHU 2023 Rates", "CBHU 2025 Rates"]
grouping_job_ids = irp_client.analysis.submit_analysis_grouping_jobs(
    [
        {
            "group_name": group_name,
            "analysis_names": analysis_names
        }
    ]
)
grouping_response = irp_client.analysis.poll_analysis_grouping_job_batch_to_completion(grouping_job_ids)
print(json.dumps(grouping_response, indent=2))

### Export Analyses to RDM

In [ ]:
analysis_names = ["CBHU Group", "CBHU 2023 Rates", "CBHU 2025 Rates"]
rdm_export_response = irp_client.rdm.export_analyses_to_rdm(
    server_name=server_name,
    rdm_name=rdm_name,
    analysis_names=analysis_names
)
print(json.dumps(rdm_export_response, indent=2))

### Delete EDM

In [ ]:
print(json.dumps(irp_client.edm.delete_edm(edm_name), indent=2))